All the Jupyter Notebooks are available at https://github.com/neuro-ml/dpipe_tutorial

# Tutorials on Deep Pipe

The tutorials introduce the library called **Deep Pipe**, which is useful for medical image analysis, including preprocessing, data augmentation, performance validation and final prediction.

## Tutorial 0: Introduction to Deep Pipe

As the first step, we get familiar with the lib, import it and try to work with one MRI image dataset from Brats 2017 competition.

### Imports:

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

# import deep pipe library
# how to install: https://github.com/neuro-ml/deep_pipe/blob/master/README.md
import dpipe

### Class `Brats2017`

Import a special class to work with the `Brats2017` dataset: 

In [2]:
from dpipe.dataset.brats import Brats2017

To create an instance of the class we have to pass two parameters: `data_path` and `metadate_rpath`.  

*The examples of the input arguments can be found at https://github.com/neuro-ml/deep_pipe/blob/master/config_examples/assets/data_source/.*

**Note:** We are using IITP machine with stored Brats dataset:
https://github.com/neuro-ml/deep_pipe/blob/master/config_examples/assets/data_source/iitp/brats.config

In [3]:
dataset = Brats2017(data_path = "/nmnt/t01-ssd/brats2017/train", metadata_rpath = "metadata.csv")

### Let's look at the available methods and attributes:

We can take a look at the loaded metadata: it has index column `id`, paths to images with different modalities `t1`, `t1ce`, ... . Also, it contains the column with the path to the target segmented image: `segm`.

In [4]:
df = dataset.df
df.head()

,cancer_type,age,survival_days,survival_class,t1,t1ce,t2,flair,segm
id,,,,,,,,,
Brats17_2013_0_1,LGG,NaN,NaN,NaN,LGG/Brats17_2013_0_1/Brats17_2013_0_1_t1.nii.gz,LGG/Brats17_2013_0_1/Brats17_2013_0_1_t1ce.nii.gz,LGG/Brats17_2013_0_1/Brats17_2013_0_1_t2.nii.gz,LGG/Brats17_2013_0_1/Brats17_2013_0_1_flair.ni...,LGG/Brats17_2013_0_1/Brats17_2013_0_1_seg.nii.gz
Brats17_2013_10_1,HGG,NaN,NaN,NaN,HGG/Brats17_2013_10_1/Brats17_2013_10_1_t1.nii.gz,HGG/Brats17_2013_10_1/Brats17_2013_10_1_t1ce.n...,HGG/Brats17_2013_10_1/Brats17_2013_10_1_t2.nii.gz,HGG/Brats17_2013_10_1/Brats17_2013_10_1_flair....,HGG/Brats17_2013_10_1/Brats17_2013_10_1_seg.ni...
Brats17_2013_11_1,HGG,29.12,150.0,0.0,HGG/Brats17_2013_11_1/Brats17_2013_11_1_t1.nii.gz,HGG/Brats17_2013_11_1/Brats17_2013_11_1_t1ce.n...,HGG/Brats17_2013_11_1/Brats17_2013_11_1_t2.nii.gz,HGG/Brats17_2013_11_1/Brats17_2013_11_1_flair....,HGG/Brats17_2013_11_1/Brats17_2013_11_1_seg.ni...
Brats17_2013_12_1,HGG,NaN,NaN,NaN,HGG/Brats17_2013_12_1/Brats17_2013_12_1_t1.nii.gz,HGG/Brats17_2013_12_1/Brats17_2013_12_1_t1ce.n...,HGG/Brats17_2013_12_1/Brats17_2013_12_1_t2.nii.gz,HGG/Brats17_2013_12_1/Brats17_2013_12_1_flair....,HGG/Brats17_2013_12_1/Brats17_2013_12_1_seg.ni...
Brats17_2013_13_1,HGG,NaN,NaN,NaN,HGG/Brats17_2013_13_1/Brats17_2013_13_1_t1.nii.gz,HGG/Brats17_2013_13_1/Brats17_2013_13_1_t1ce.n...,HGG/Brats17_2013_13_1/Brats17_2013_13_1_t2.nii.gz,HGG/Brats17_2013_13_1/Brats17_2013_13_1_flair....,HGG/Brats17_2013_13_1/Brats17_2013_13_1_seg.ni...


To get the indices we can use the property **ids**:

In [5]:
dataset.ids[:4]

['Brats17_2013_0_1',
 'Brats17_2013_10_1',
 'Brats17_2013_11_1',
 'Brats17_2013_12_1']

Names of columns with modalities in the dataset:

In [6]:
dataset.modality_cols

['t1', 't1ce', 't2', 'flair']

The number of channel (modalities) in the given dataset, returns `len(['t1', 't1ce', 't2', 'flair'])`

In [7]:
dataset.n_chans_image

4

Name of target column:

In [8]:
dataset.target_col

'segm'

### Load the images:

We can load the images for a certain patient using the `load_image(patient_id)` method:

In [9]:
patient_id = 'Brats17_2013_12_1'
img = dataset.load_image(patient_id)
print('Shape of the image: ', img.shape)

Shape of the image:  (4, 240, 240, 155)


To visualize the images we will use some awesome function `slice3d`. Let's firstly import it:

In [10]:
from dpipe.medim.visualize import slice3d

We can plot all modalities as follows:

In [11]:
slice3d(*img, axis=-1, fig_size=5, max_columns=None, colorbar=False)

A Jupyter Widget

Or plot only certain ones:

In [12]:
slice3d(img[0], img[2], axis=-1, fig_size=4, max_columns=None, colorbar=False)

A Jupyter Widget

### Load the segmented images:

We can load the segmented image by the `load_segm(patient_id)` method:

In [13]:
segm_img = dataset.load_segm(patient_id)
print('Shape of the segmented image: ', segm_img.shape)

Shape of the segmented image:  (240, 240, 155)


In [14]:
# the number of diffent segmentation madalities (0 - black)
dataset.n_chans_segm

4

The segmentation is normally coded by integer numbers, i.e. an integer is assigned for each voxel. For example, in Brats2017, there are 4 such integers: (0) - healthy tissue, (1) - dead core of cancer, (2) - living core of cancer, (3) - inflamed tissue. Take a look:

In [15]:
slice3d(segm_img, fig_size=5, colorbar=True)

A Jupyter Widget

The task is to find three binary masks: the whole cancer (1+2+3), core of cancer (1+2) and inflamed tissue (3). The quality metrics are computed using each of the masks independently. Thus, we might want to get these masks from the segmented image.

We can load the multimodal segmented image by the `load_msegm(patient_id)` method. It returns multimodal segmentation of shape [n_chans_msegm, x, y, z].

In [16]:
msegm_img = dataset.load_msegm(patient_id)
print('Shape of the multimodal segmented image: ', msegm_img.shape)

Shape of the multimodal segmented image:  (3, 240, 240, 155)


In [17]:
slice3d(*msegm_img, fig_size=5, colorbar=True)

A Jupyter Widget

To convert images coded by ints into separate binary masks, the segm2msegm matrix is used. The rows correspond to segmented image, whereas the columns - to multimodal segmented image:

In [18]:
dataset.segm2msegm_matrix

array([[False, False, False],
       [ True,  True, False],
       [ True, False, False],
       [ True,  True,  True]], dtype=bool)

We can also transfer arbitrary image into binary masks using `segm2msegm` method:

In [19]:
msegm_img = dataset.segm2msegm(segm_img)
print('Shape: ', dataset.segm2msegm(segm_img).shape)

Shape:  (3, 240, 240, 155)
